<a href="https://colab.research.google.com/github/batra-ayush/PyTorch-Examples/blob/main/Iris_with_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,TensorDataset

In [2]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(len(data), dtype=torch.long)

labels[iris.species=='versicolor'] =1
labels[iris.species=='virginica'] = 2
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [4]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels,test_size=0.2)

len(data)

150

In [5]:
train_data = TensorDataset(torch.Tensor(train_data),torch.Tensor(train_labels))
test_data = TensorDataset(torch.Tensor(test_data),torch.Tensor(test_labels))

In [6]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)
test_loader  = DataLoader(test_data)

In [7]:
test_data.tensors[0].shape[0]

30

In [14]:
class theClassforIris(nn.Module):
  def __init__(self,dropoutrate):
    super().__init__()
    self.input = nn.Linear(4,12)
    self.hidden = nn.Linear(12,12)
    self.output = nn.Linear(12,3)

    self.dr=dropoutrate

  def forward(self,x):
    x = self.input(x)
    x = F.relu(x)
    x = F.dropout(x,p=self.dr,training=self.training)
    x = self.hidden(x)
    x  = F.relu(x)
    x = F.dropout(x,p=self.dr,training=self.training)
    x = self.output(x)
    return x

In [15]:
def createANewModel(dropoutrate):
  AnnIris = theClassforIris(dropoutrate)
  lossfun = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(AnnIris.parameters(),lr=.005)
  return AnnIris, lossfun, optimizer

In [49]:
numepochs = 400

def trainTheModel():
  trainAcc = []
  testAcc = []

  for epochi in range(numepochs):
    AnnIris.train()
    batchAcc = []
    for X,Y in train_loader:
      yHat = AnnIris(X)
      loss = lossfun(yHat,Y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      batchAcc.append(100*torch.mean((torch.argmax(yHat,axis=1) == Y).float()).item())
    trainAcc.append(np.mean(batchAcc))
  AnnIris.eval()
  X,Y = next(iter(test_loader))
  predlabel = torch.argmax(AnnIris(X),axis=1)
  testAcc.append(100*torch.mean((predlabel == Y).float()).item())
  return trainAcc, testAcc

In [50]:
dropoutrate = .05

AnnIris,lossfun, optimizer = createANewModel(dropoutrate)
trainacc,testacc = trainTheModel()

In [51]:
np.mean(trainacc)

84.814453125

In [48]:
testacc

[100.0]

In [ ]:
testAcc = []
for X,y in test_loader:
  ypred = AnnIris(X)
  testAcc.append((100*torch.mean((torch.argmax(ypred,axis=1) == y).float())).item())



In [ ]:
testAcc

[100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 0.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0]

In [ ]:
accpct = 100*(torch.mean(accuracy.float()))
accpct